In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy = 1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'APY_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'att'                  #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import TripletNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(64,1600,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)
triplet_network = TripletNetwork(metric_network)  #metric learning   
triplet_network.cuda(GPU)
attribute_network.cuda(GPU)
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
attribute_network_optim=torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
attribute_network_scheduler = StepLR(attribute_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float()) # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext).view(-1, class_num)
        #relations = F.cos(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
        #rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        #total_rewards += np.sum(rewards)
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    
    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
#writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)
    attribute_network_scheduler.step(episode)
    

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    batch_attributes_negative = torch.empty_like(batch_attributes)
    
    
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=batch_labels.numpy()[i])[0], replace=True)
        batch_features_negative[i] = batch_features[pick]
    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    
    
    for i in range(batch_attributes_negative.size()[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=batch_labels.numpy()[i])[0], replace=True)
        batch_attributes_negative[i] = batch_attributes[pick]
    
    
    #print(batch_attributes.size())
    #print(batch_attributes_negative.size())
    
    
    #batch_attributes_negative = torch.from_numpy(batch_features_negative)
    batch_attributes_negative = Variable(batch_attributes_negative).cuda(GPU).float()  # 32*2048
    
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    #print((batch_attributes_negative).size())
    #print(attribute_network(batch_attributes_negative).size())
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(attribute_network(batch_attributes), batch_features, batch_features_negative)
    dista2, distb2, embedded_x2, embedded_y2, embedded_z2 = triplet_network(batch_features,
                                                                            attribute_network(batch_attributes), 
                                                                            attribute_network(batch_attributes_negative))
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target) + criterion(dista2, distb2, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet #+ Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    attribute_network.zero_grad()
    metric_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    attribute_network_optim.step()
    
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss)
        #writer.add_scalar('data/loss', loss, episode)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(32), attributes)
        gzsl_seen_accuracy = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(32), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()


training...
episode: 1000 loss tensor(0.3310, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.31798113161812996
gzsl: unseen=0.0966 , seen=0.5379 , h=0.1639
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.2963, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3220003131034348
gzsl: unseen=0.1382 , seen=0.5617 , h=0.2218
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.0975, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.34754912428545376
gzsl: unseen=0.1846 , seen=0.5785 , h=0.2799
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.1539, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class 

zsl: 0.3222878684929014
gzsl: unseen=0.2026 , seen=0.6499 , h=0.3089
____________________________________________________________________________________________________
episode: 29000 loss tensor(0.0287, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3311594178362151
gzsl: unseen=0.2194 , seen=0.6494 , h=0.3280
____________________________________________________________________________________________________
episode: 30000 loss tensor(0.0087, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3160419382041126
gzsl: unseen=0.2157 , seen=0.6444 , h=0.3232
____________________________________________________________________________________________________
episode: 31000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3204718418378575
gzsl: unseen=0.2102 , seen=0.6564 , h=0.3184
______________________________________

class num: 32
class num: 32
zsl: 0.3131882506926812
gzsl: unseen=0.2017 , seen=0.6789 , h=0.3110
____________________________________________________________________________________________________
episode: 57000 loss tensor(0.0063, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.29803732399735355
gzsl: unseen=0.1921 , seen=0.6867 , h=0.3002
____________________________________________________________________________________________________
episode: 58000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3029675824215481
gzsl: unseen=0.1922 , seen=0.6950 , h=0.3011
____________________________________________________________________________________________________
episode: 59000 loss tensor(0.0109, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.30125313698325995
gzsl: unseen=0.1785 , seen=0.7004 , h=0.2846
________

class num: 32
class num: 32
zsl: 0.31742082324993587
gzsl: unseen=0.1888 , seen=0.7016 , h=0.2976
____________________________________________________________________________________________________
episode: 85000 loss tensor(0.0081, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.30926629185122456
gzsl: unseen=0.1792 , seen=0.7171 , h=0.2868
____________________________________________________________________________________________________
episode: 86000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3016140130132387
gzsl: unseen=0.1852 , seen=0.6991 , h=0.2928
____________________________________________________________________________________________________
episode: 87000 loss tensor(0.0005, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.2928542602876409
gzsl: unseen=0.1570 , seen=0.7115 , h=0.2572
________

episode: 112000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.33469293535567424
gzsl: unseen=0.2069 , seen=0.7080 , h=0.3202
____________________________________________________________________________________________________
episode: 113000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.33679810985981207
gzsl: unseen=0.2032 , seen=0.7145 , h=0.3164
____________________________________________________________________________________________________
episode: 114000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.34114242734462136
gzsl: unseen=0.2110 , seen=0.7093 , h=0.3252
____________________________________________________________________________________________________
episode: 115000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 

...

episode: 392000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.34188720388818483
gzsl: unseen=0.1868 , seen=0.7281 , h=0.2973
____________________________________________________________________________________________________
episode: 393000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3277032938362397
gzsl: unseen=0.1775 , seen=0.7203 , h=0.2848
____________________________________________________________________________________________________
episode: 394000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.34378808027883045
gzsl: unseen=0.1878 , seen=0.7314 , h=0.2988
____________________________________________________________________________________________________
episode: 395000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 3

episode: 420000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3448999707621656
gzsl: unseen=0.1860 , seen=0.7141 , h=0.2951
____________________________________________________________________________________________________
episode: 421000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.33788355110569773
gzsl: unseen=0.1691 , seen=0.7235 , h=0.2742
____________________________________________________________________________________________________
episode: 422000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3485030540535434
gzsl: unseen=0.1742 , seen=0.7320 , h=0.2814
____________________________________________________________________________________________________
episode: 423000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32

episode: 448000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.327013890675307
gzsl: unseen=0.1630 , seen=0.7221 , h=0.2659
____________________________________________________________________________________________________
episode: 449000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3141933234178213
gzsl: unseen=0.1533 , seen=0.7279 , h=0.2532
____________________________________________________________________________________________________
episode: 450000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3267913655090804
gzsl: unseen=0.1618 , seen=0.7198 , h=0.2642
____________________________________________________________________________________________________
episode: 451000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
c

episode: 476000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3503040037264073
gzsl: unseen=0.1782 , seen=0.7185 , h=0.2856
____________________________________________________________________________________________________
episode: 477000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.3470915878474015
gzsl: unseen=0.1782 , seen=0.7308 , h=0.2866
____________________________________________________________________________________________________
episode: 478000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.34598908624458646
gzsl: unseen=0.1717 , seen=0.7186 , h=0.2772
____________________________________________________________________________________________________
episode: 479000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 12
class num: 32

KeyboardInterrupt: 